# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [22]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)
list_gen = []

for sqrt_value in generator:
    print(sqrt_value)
    list_gen.append(sqrt_value)

print(sum(list_gen))



1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [7]:
import dlt

In [6]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [15]:
# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = generators_pipeline.run(people_1(),
                                 table_name="people_1",
                                 write_disposition="replace")

print(info)

info = generators_pipeline.run(people_2(),
                                table_name="people_2",
                                write_disposition="replace")

Pipeline dlt_ipykernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/diwal/Library/Mobile Documents/com~apple~CloudDocs/Work/01_Cours/07_data_engineering_zoomcamp/de_zoomcamp2024/workshop_01/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708192283.119673 is LOADED and contains no failed jobs


In [21]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n people_1 and people_2 table below:")

all = conn.sql("SELECT * FROM people_1").df()
print(all)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_1            │
│ people_2            │
└─────────────────────┘




 people_1 and people_2 table below:
   id      name  age    city       _dlt_load_id         _dlt_id
0   1  Person_1   26  City_A  1708192283.119673  7NDE6AdSNTVPsA
1   2  Person_2   27  City_A  1708192283.119673  ed6X8pZHh3bCeg
2   3  Person_3   28  City_A  1708192283.119673  QDeLb8j4syGsBw
3   4  Person_4   29  City_A  1708192283.119673  jv33LgESo0iW6w
4   5  Person_5   30  City_A  1708192283.119673  DhWB+xwHZj4g8g


In [34]:
sum1 + sum2

,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [56]:
# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline_1 = dlt.pipeline(destination='duckdb', dataset_name='generators')

infos = generators_pipeline_1.run(people_1(),
                                 table_name="people",
                                 primary_key="ID",
                                 write_disposition="replace")

infos = generators_pipeline_1.run(people_2(),
                                 table_name="people",
                                 primary_key="ID",
                                 write_disposition="merge")

In [52]:
print(infos)

Pipeline dlt_ipykernel_launcher load step completed in 0.33 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/diwal/Library/Mobile Documents/com~apple~CloudDocs/Work/01_Cours/07_data_engineering_zoomcamp/de_zoomcamp2024/workshop_01/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708249583.002086 is LOADED and contains no failed jobs


In [65]:
conn = duckdb.connect(f"{generators_pipeline_1.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{generators_pipeline_1.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n people_1 and people_2 table below:")

table = conn.sql("""SELECT * FROM people """)

#as p1
#left join 
#people_2 as p2 on p1.ID = p2.ID""").df()
print(table)

table_sum_age = conn.sql("""SELECT SUM(age) FROM people """).fetchall()
print(table_sum_age)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_1            │
│ people_11           │
│ people_2            │
│ people_22           │
└─────────────────────┘




 people_1 and people_2 table below:
┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708249972.1997828 │ 3fNi/XMpOcEj3Q │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708249972.1997828 │ YRx6kuMkqujDMA │ NULL       │
│     8 │ Person_8 │    38 │ City_B  │ 1708249972.7986088 │ r+UJD0EfXDJhdQ │ Job_8      │
│     3 │ Person_3 │    33 │ City_B  │ 1708249972.7986088 │ 4J+oHudmGLRjuw │ Job_3      │
│     6 │ Person_6 │    36 │ City_B  │ 1708249972.7986088 │ 6YaBm588IDOcuQ │ Job_6      │
│     5 │ Person_5 │    35 │ City_B  │ 1708249972.7986088 │ wq/ODhdq/b6KwA │ Job_5      │
│     7 │ Person_7 │    37 │ City_B  │ 1708249972.7986088 │ y

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX